In [127]:
import os
import glob
import cv2

#### Godot TSCN Generation Helpers

In [155]:
def gd_scene(opts):
    return "[gd_scene load_steps={load_steps} format=2]\n".format(**opts)

def ext_resource(opts):
    return '[ext_resource path="{path}" type="Texture" id={id}]\n'.format(**opts)

def tile_markup(opts):
    markup  = '{tile_id}/name = "{tile_name}"\n'
    markup += '{tile_id}/texture = ExtResource( {resource_id} )\n'
    markup += '{tile_id}/tex_offset = Vector2( 0, 0 )\n'
    markup += '{tile_id}/modulate = Color( 1, 1, 1, 1 )\n'
    markup += '{tile_id}/region = Rect2( {region_x_offset}, {region_y_offset}, 32, 32 )\n'
    markup += '{tile_id}/tile_mode = 0\n'
    markup += '{tile_id}/occluder_offset = Vector2( 0, 0 )\n'
    markup += '{tile_id}/navigation_offset = Vector2( 0, 0 )\n'
    markup += '{tile_id}/shapes = [  ]\n'
    markup += '{tile_id}/z_index = 0\n'
    return markup.format(**opts)

def sub_resource(opts):
    return '[sub_resource type="{type}" id={id}]\n'.format(**opts)

def node(opts):
    markup  = '[node name="{name}" type="{type}"]\n'
    markup += 'tile_set = SubResource( 1 )\n'
    markup += 'cell_size = Vector2( 32, 32 )\n'
    markup += 'format = 1\n'
    
    return markup.format(**opts)

#### Generating File

In [190]:
def generate_tileset(target_file, target_files,tilemap_name):
    ext_resources = []
    tiles         = []
    major_tile_id = 1
    for resource_id, tilepath in enumerate(target_files):
        resource_id += 1
        basename    = os.path.basename(tilepath).lower()
        img         = cv2.imread(tilepath)
        num_horz    = int(img.shape[1] / 32)
        num_vert    = int(img.shape[0] / 32)
        ext_resources.append(ext_resource({
            "path": "res://tiles/type_1/%s" % os.path.basename(tilepath),
            "id":   resource_id
        }))

        sub_tile_id = 1
        for x in range(num_horz):
            for y in range(num_vert):
                tiles.append(tile_markup({
                    "tile_id": major_tile_id,
                    "tile_name": "%s_%04d" % (basename, sub_tile_id),
                    "resource_id": resource_id,
                    "region_x_offset": x * 32,
                    "region_y_offset": y * 32
                }))
                sub_tile_id += 1
                major_tile_id += 1
                
    with open(target_file, "w") as f:
        f.write(gd_scene({"load_steps": len(ext_resources)}))
        for x in ext_resources: f.write(x)
        f.write(sub_resource({"type":"TileSet", "id": 1}))
        for x in tiles: f.write(x)
        f.write(node({"name":tilemap_name, "type":"TileMap"}))

In [191]:
generate_tileset(
    target_file  = "E:/__godot-workspace/net-2d-topdown-test/tiles/new_tiles.tscn",
    target_files = glob.glob("E:/__godot-workspace/net-2d-topdown-test/tiles/type_1/*Base*.png"),
    tilemap_name = "new_tiles"
)